In [7]:
from src.genre_classification.F_Basic_Models import Basic_Models
from src.genre_classification.F_Dataset_Downloader import Dataset_Downloader
import pandas as pd

In [8]:
dataset_downloader = Dataset_Downloader()
train_path, test_path = dataset_downloader()

In [9]:
train_data = pd.read_csv(train_path.resolve())
test_data = pd.read_csv(test_path.resolve())


In [10]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------------- -------------------------- 4.2/12.8 MB 31.5 MB/s eta 0:00:01
     ----------------------------------- --- 11.5/12.8 MB 34.4 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 29.7 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


c:\Users\harry\Documents\Proyectos-URJC\Python\NLP_II\NLP_II_Practica1\p1_venv\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\harry\Documents\Proyectos-URJC\Python\NLP_II\NLP_II_Practica1\p1_venv\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\harry\Documents\Proyectos-URJC\Python\NLP_II\NLP_II_Practica1\p1_venv\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [11]:
x_train, y_train = train_data.drop(columns=["genre"]), train_data["genre"]
x_test, y_test = test_data.drop(columns=["genre"]), train_data["genre"]

In [ ]:
rn_forest = Basic_Models(model_type="Random_Forest")

rn_forest.fit(x_train, y_train)

y_hat = rn_forest.transform(x_test)

print(y_hat, y_test)

Modelo de spaCy 'en_core_web_sm' cargado.
